## 爬取公開資料觀測站 房地產資料

資料分為下面幾種

x_lvr_land_a：房屋買賣交易

x_lvr_land_b：新成屋交易

x_lvr_land_c：租房交易

其中 x 代表每個縣市，也就是你身份證字號的開頭，例如 台北為 a，新北為 f，以此類推

In [ ]:
from function.crawl import crawl_house,data_select
crawl_house(2015,2022)
data_select(select = "a_lvr_land_a.csv") # out= csv # still dirty

## Shopping mall 等其他資料 Crawler

In [ ]:
# Get Shopping mall
import requests
import pandas as pd
import time
import random
headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}
li = []
main ='https://www.tripadvisor.com.tw/'
for i in range(0,450,30):
    time.sleep(3 + random.uniform(0,5))
    url = 'https://www.tripadvisor.com.tw/Attractions-g293913-Activities-c26-oa'+str(i)+'-Taipei.html'
    res = requests.get(url,headers=headers)
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(res.text, 'html.parser')
    # 所有的超連結
#     a_tags = soup.find_all('div',attrs ={"class":'bUshh o csemS'})
    a_tags = soup.find_all('div',attrs ={"class":'fVbwn cdAAV cagLQ eZTON'})
    for tag in a_tags:
    #   # 輸出超連結的文字
#         st = tag.get_text()
#         st = st.split('. ')[1]
#         li.append(st)
        # 輸出子網路
        li.append('https://www.tripadvisor.com.tw'+tag.find('a').get('href'))

In [ ]:
# pd.Series(li).to_csv("tapei_mall_link.csv")
# li= pd.read_csv("tapei_mall_link.csv").iloc[:,1]

# addr_nm = pd.read_csv("Tapei_shoppingmall_name.csv").iloc[:,1]
# addr = pd.Series(text)
# df = pd.concat([addr_nm,addr,li],axis=1)
# df = df.iloc[:,1:]
# df.columns = ['name','addr','link']
# df.to_csv("all_mall.csv")

In [ ]:
## 檢查資料正確性
# li = ['W"xmlns:og="http://opengraphprotocol.org/schema/"><head><meta','_GB-','tahttp-equiv="content-type"','台北No.','台北No.366,',]
# get = [ t for t in df.addr if t not in li]
# df = df.query('addr in @get')
# df = df.reset_index(drop=True)
# df = df.iloc[:,1:]
# df.to_csv("shoppingmall_final.csv")

In [ ]:
# 爬蟲嘗試
text =[]
import gc
gc.collect()
def get_address(res): # 只能擷取到部分地址
    a= res.text.split('地址</div></div>')[-1]
    a = a.split('</div>')[0]
    b = a.split(' ')[2]
    a = a.split(' ')[3]
    a = b[-2:]+a
    gc.collect()
    return a

for i in li:
    time.sleep(3 + random.uniform(0,5))
    res = requests.get(i,headers=headers)
    text.append(get_address(res))
    
final = [t for t in text if t not in li]
pd.Series(final).to_csv('Shoppingmall_148.csv')

In [ ]:
# 抓台北School address
import requests
import pandas as pd
df = pd.read_html('https://www.doe.gov.taipei/News_Content.aspx?n=026199D6B5AC5A6A&sms=DDAA880EFAADF5F3&s=7472A783D2FDD6F7')

df = df[1]
def remove_brackets(s):
    ind = s.find(']')
    s = s[ind+1:]
    return s

df.地址 = df.地址.apply(remove_brackets)
df.to_csv('Tapei_school_address.csv')

In [ ]:
## 抓取捷運地址
import requests
import pandas as pd
from bs4 import BeautifulSoup

# crawl 站牌名
url = 'https://web.metro.taipei/apis/metrostationapi/stationsidlabellist'
headers ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
         'Accept':'application/json, text/plain, */*',
         'Host': 'web.metro.taipei',
         'Origin': 'https://web.metro.taipei',
         'Referer': 'https://web.metro.taipei/pages/tw/roadmap/station',
         'Sec-Fetch-Mode': 'cors',
         'Sec-Fetch-Site': 'same-origi'
        }

payload ={'Lang': "tw",'LineID': "0"}

r = requests.post(url,data=payload,headers=headers)
pd.read_json(r.text,encoding='utf-8')
df = pd.read_json(r.text)

df['crawl']= df.StationLabel+" "+df.StationName

In [ ]:
## 抓捷運站牌名
import time
import random
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
options = webdriver.ChromeOptions()
def crawl_MRTaddress(t):
    time.sleep(8 + random.uniform(0,5))
    options.add_argument("headless")

    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options) #webdriver.Chrome('./chromedriver',options=options) #
    
    browser.get("https://www.metro.taipei/cp.aspx?n=91974F2B13D997F1")
    browser.switch_to.frame('route')
    sr = browser.find_element_by_id("menulistsingle")
#     sr = browser.find_element_by_xpath('//*[@id="menulistsingle"]')
    time.sleep(2 + random.uniform(0,2 )) 
    sr.send_keys(t) 
    time.sleep(2 + random.uniform(0,3 )) 
    confirm = browser.find_element_by_xpath('/html/body/app-root/app-roadmap/app-menuline/div/div[2]')
    confirm.click()
    time.sleep(2 + random.uniform(0,5 ))
    browser.switch_to_default_content()
    browser.switch_to.frame('route')
    text_loc = browser.find_element_by_css_selector('body > app-root > app-station > div > table > tr:nth-child(2) > td > span')
    text = text_loc.text
    browser.quit()
    return text
li = []
for t in df.crawl:
    print(t)
    li.append(crawl_MRTaddress(t))


rt = pd.Series(li)    
def convertString(k):
    if len(k)>=40:
        k = rt.iloc[132]
        ind = k.find("\n")
        k = k[ind+1:]
        ind1= k.find('：')
        k = k[ind1+1:]
    import re
    k = re.sub(r'^[0-9]+', '', k)
    return k

rt = rt.apply(convertString)
rt.to_csv("MRT_address.csv")

BR01 動物園


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: use driver.switch_to.default_content instead


BR02 木柵
BR03 萬芳社區
BR04 萬芳醫院
BR05 辛亥
BR06 麟光
BR07 六張犁
BR08 科技大樓
BR09 大安
BR10 忠孝復興
BR11 南京復興
BR12 中山國中
BR13 松山機場
BR14 大直
BR15 劍南路
BR16 西湖
BR17 港墘
BR18 文德
BR19 內湖
BR20 大湖公園
BR21 葫洲
BR22 東湖
BR23 南港軟體園區
BR24 南港展覽館
R02 象山
R03 台北101/世貿
R04 信義安和
R05 大安
R06 大安森林公園
R07 東門
R08 中正紀念堂
R09 台大醫院
R10 台北車站
R11 中山
R12 雙連
R13 民權西路
R14 圓山
R15 劍潭
R16 士林
R17 芝山
R18 明德
R19 石牌
R20 唭哩岸
R21 奇岩
R22 北投
R22A 新北投
R23 復興崗
R24 忠義
R25 關渡
R26 竹圍
R27 紅樹林
R28 淡水
G01 新店
G02 新店區公所
G03 七張
G03A 小碧潭
G04 大坪林
G05 景美
G06 萬隆
G07 公館
G08 台電大樓
G09 古亭
G10 中正紀念堂
G11 小南門
G12 西門
G13 北門
G14 中山
G15 松江南京
G16 南京復興
G17 台北小巨蛋
G18 南京三民
G19 松山
O01 南勢角
O02 景安
O03 永安市場
O04 頂溪
O05 古亭
O06 東門
O07 忠孝新生
O08 松江南京
O09 行天宮
O10 中山國小
O11 民權西路
O12 大橋頭
O13 台北橋
O14 菜寮
O15 三重
O16 先嗇宮
O17 頭前庄
O18 新莊
O19 輔大
O20 丹鳳
O21 迴龍
O50 三重國小
O51 三和國中
O52 徐匯中學
O53 三民高中
O54 蘆洲
BL01 頂埔
BL02 永寧
BL03 土城
BL04 海山
BL05 亞東醫院
BL06 府中
BL07 板橋
BL08 新埔
BL09 江子翠
BL10 龍山寺
BL11 西門
BL12 台北車站
BL13 善導寺
BL14 忠孝新生
BL15 忠孝復興
BL16 忠孝敦化
BL17 國父紀念館
BL18 市政府
BL19 永春
BL20 後山埤
BL21 昆陽
BL22 南港
BL23 南港展

## 地址轉換
* 此程式碼有寫多個版本，此為版本1

In [ ]:
import os
import pandas as pd
import gc 
import csv
from function.distance import get_coordinate
import time
import random

def main(df,path,t):
    # check mem info
    import psutil 
    proc = psutil.Process(os.getpid())
    mem0 = proc.memory_info().rss
    
    # main 
    with open(path,'w', newline='') as csvfile:
        writer = csv.writer(csvfile)   
        writer.writerow(['Index', 'address','0']) # 寫入一列資料
        gc.collect() # release mem
        for col in df: # test df.head(2)
            i = 3
            while i>0:
                try: 
                    time.sleep(8 + random.uniform(0,10 ))
    #                 print(t,col)
                    writer.writerow([t,col,get_coordinate(col)])
                    i-=3
                    t+=1 # record index
                    gc.collect() # release mem
                except:
                    print(t,'err')
                    time.sleep(60)
                    i-=2
            if i==-1:
                t+=1 # record index
                writer.writerow([t, col,"nan"])  
                gc.collect()

            if t%2000 == 0: # 每2000筆印一筆資料
                print(t,'ok')

### 之所以拆開不用程式loop是因為實際會在某一段落被鎖IP，所以拆開來跑會較方便

In [ ]:
# read hospital address name
hos_dir = os.path.join('output','hospital.csv')
df = pd.read_csv(hos_dir)
df = df['地址']
gc.collect()

# initialize
t = 0
pt = os.path.join('output','hospital_addr.csv')
main(df.iloc[t:],pt,t)

In [ ]:
# read house address name
new_dir = os.path.join('house_data','new_data.csv')
df = pd.read_csv(new_dir)

df = df['土地位置建物門牌']
gc.collect()

# initialize
t = 0
path = os.path.join('output','house','house_addr'+str(t)+'.csv')
gc.collect() # release mem

main(df.iloc[t:],path,t)

In [ ]:
## read shop address name
df = pd.read_csv('Shoppingmall_addr_148.csv')

df = df['0']
gc.collect() # release mem

# initialize
t = 0

path = os.path.join('output','shoppingmall','shop_addr'+str(t)+'.csv')
gc.collect() # release mem

main(df.iloc[t:],path,t)

In [ ]:
## Taipei school address

df = pd.read_csv('Tapei_school_address.csv')
df = df['地址']
gc.collect() # release mem

# initialize
t = 0
gc.collect() # release mem

path = os.path.join('output','school','tpe_school_addr'+str(t)+'.csv')
gc.collect() # release mem

main(df.iloc[t:],path,t)

In [ ]:
## MRT address
df = pd.read_csv('MRT_address.csv')
df = df['0']
gc.collect() # release mem

# initialize
t = 0
gc.collect() # release mem

path = os.path.join('output','mrt','mrt_addr'+str(t)+'.csv')
gc.collect() # release mem

main(df.iloc[t:],path,t)

## Train address
df = pd.read_csv('train_addr.csv')
df = df['0']
gc.collect() # release mem

# initialize
t = 0
gc.collect() # release mem

path = os.path.join('output','train','train_addr'+str(t)+'.csv')
gc.collect() # release mem

main(df.iloc[t:],path,t)